# Playing around with the Spotify API

In [34]:
import pandas as pd
import numpy as np
import os
import sys
import spotipy
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation


In [2]:
'''
define the API connection - usually log in from here as well...
'''

spotify=spotipy.Spotify()


In [3]:
'''example with finding a users albums... '''
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP' #try and make a function
#that returns this 'uri' given artist name.

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Live In London
Birdy
Birdy
Birdy
Birdy (Spotify Exclusive)
Birdy (Deluxe Version)
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)


In [28]:
''' 
luke i changed ur function down below the index 1 was wrong so i put it as  0 and it giving
way better results, chur...
'''
def get_uri(artist):
    results = spotify.search(q = "artist:" + artist, type = "artist")
    artist_id = results['artists']['items'][0]['uri'] # changed from '1' to '0' ??
   
    return(artist_id)

get_uri("Beyonce")
#A function that returns artist 'uri' as a string. 
#By most matching (first result)

'spotify:artist:6vWDO969PvNqNYHIOW5v0m'

In [33]:
''' lets try get related song '''

artst = 'Young Thug'

related = spotify.artist_related_artists(get_uri(artst))

for artist in related['artists']:
    print('  ', artist['name'])
    print('  ', artist['popularity'])
    print('  ', artist['followers']['total'])



   Rich Homie Quan
   76
   555307
   2 Chainz
   87
   949694
   Waka Flocka Flame
   73
   422060
   Young Thug
   86
   795434
   Yo Gotti
   80
   415369
   Migos
   85
   614731
   French Montana
   81
   434759
   Travis Scott
   86
   646535
   Gucci Mane
   89
   536596
   Meek Mill
   81
   747859
   K CAMP
   70
   301339
   Young Dolph
   71
   109900
   Chief Keef
   72
   436949
   A$AP Ferg
   76
   395207
   Jeezy
   77
   496811
   Juicy J
   80
   519363
   21 Savage
   86
   437196
   Rick Ross
   82
   886406
   DJ Khaled
   79
   720479
   DJ Drama
   67
   149708
